# MLflow tutorial
In the first week, we'll focus on MLflow that helps you track your ML experiments. 

<img src="../images/overview-mlflow-focus.jpg" width=800>

As shown in the figure above, the MLOps platform provides an MLflow service that stores ML model training information to a PostgreSQL database and training artifacts to a MinIO storage service. 

MLflow provides a [Python client](https://mlflow.org/docs/latest/python_api/index.html) for communicating with an MLflow service. For example, we can use the MLflow Python client to start an *MLflow run*, which is an execution of an ML training script:
```python
with mlflow.start_run():
    model = ElasticNet(alpha=..., l1_ratio=...)
    model.fit(train_x, train_y)
```
*MLflow runs* are organized into *MLflow experiments*. An MLflow experiment can be seen as a logical unit of one or more MLflow runs. For example, there can be an MLflow experiment for training an ElasticNet model, and there can be multiple MLflow runs under this experiment for exploring different hyperparameters and/or training datasets.

When starting  an MLflow run, we can record the relevant information, such as the configured hyperparameters and custom evaluation metrics. After the run is completed, we can also upload the produced model artifact to MLflow:
```python
with mlflow.start_run():
    model = ElasticNet(alpha=..., l1_ratio=...)
    model.fit(train_x, train_y)
    mlflow.log_param("alpha", ...)
    mlflow.log_param("l1_ratio", ...)
    mlflow.log_metric("rmse", ...)
    mlflow.sklearn.log_model(model, ...)
```
There will be a complete example later. 

More reading material: [MLflow docs](https://mlflow.org/docs/latest/index.html)

# MLflow example
In this example, we'll use sklearn to train a simple ElasticNet model that predicts red wine quality given some chemical attributes. The information of dataset used in this example can be found [here](https://archive.ics.uci.edu/dataset/186/wine+quality). 

### Create an MLflow run
The following code snippet exemplifies how to use the MLflow Python client to record training parameters and evaluation metrics as well as upload the trained model artifact to the MLflow service.

In [1]:
import os
import logging

import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

# Set an environmental variable named "MLFLOW_S3_ENDPOINT_URL" so that MLflow client knows where to save artifacts.
# The MinIO storage service can be accessed via http://mlflow-minio.local
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://mlflow-minio.local"

# Configure the credentials needed for accessing the MinIO storage service.
# "AWS_ACCESS_KEY_ID" has been configured in a ComfigMap and "AWS_SECRET_ACCESS_KEY" in a Secret in your K8s cluster when you set up the MLOps platform
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

MLFLOW_TRACKING_URI = "http://mlflow-server.local" # This is the URL of the MLflow service
MLFLOW_EXPERIMENT_NAME = "mlflow-minio-test"


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    return rmse


def main():
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )

    data = pd.read_csv(csv_url, sep=";")

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    
    # Just use hard-coded hyperparameters
    alpha = 0.5
    l1_ratio = 0.5

    logger.info(f"Using MLflow tracking URI: {MLFLOW_TRACKING_URI}")

    # Configure the MLflow client to connect to the MLflow service
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

    logger.info(f"Using MLflow experiment: {MLFLOW_EXPERIMENT_NAME}")
    mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

    with mlflow.start_run() as run:
        print("MLflow run_id:", run.info.run_id) # Each MLflow Run has a unique identifier called run_id

        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)

        logger.info("Fitting model...")

        lr.fit(train_x, train_y)

        logger.info("Finished fitting")

        predicted_qualities = lr.predict(test_x)

        rmse = eval_metrics(test_y, predicted_qualities)

        logger.info("Elasticnet model (alpha=%f, l1_ratio=%f):" %
                    (alpha, l1_ratio))
        logger.info("  RMSE: %s" % rmse)


        logger.info("Logging parameters to MLflow")
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)

        logger.info("Logging trained model")
        artifact_name = "model"
        mlflow.sklearn.log_model(
            lr, artifact_name, registered_model_name="ElasticnetWineModel")
        print("The S3 URI of the logged model:", mlflow.get_artifact_uri(artifact_path=artifact_name))

In [2]:
main()

INFO:__main__:Using MLflow tracking URI: http://mlflow-server.local
INFO:__main__:Using MLflow experiment: mlflow-minio-test
2024/03/25 11:43:04 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-minio-test' does not exist. Creating a new experiment.
INFO:__main__:Fitting model...
INFO:__main__:Finished fitting
INFO:__main__:Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
INFO:__main__:  RMSE: 0.7931640229276851
INFO:__main__:Logging parameters to MLflow
INFO:__main__:Logging trained model


MLflow run_id: 4b3be33ec61245efaa40f76f36a43abb


INFO:botocore.credentials:Found credentials in environment variables.
Successfully registered model 'ElasticnetWineModel'.
2024/03/25 11:43:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetWineModel, version 1


The S3 URI of the logged model: s3://mlflow/6/4b3be33ec61245efaa40f76f36a43abb/artifacts/model


Created version '1' of model 'ElasticnetWineModel'.


Expected output:

```text
INFO:__main__:Using MLflow tracking URI: http://mlflow-server.local
INFO:__main__:Using MLflow experiment: mlflow-minio-test
2023/08/16 13:45:22 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-minio-test' does not exist. Creating a new experiment.
(More logs...)

MLflow run_id: fca4fffeab3d44e98ad2584f9f32a45
Successfully registered model 'ElasticnetWineModel'.
The S3 URI of the logged model: s3://mlflow/7/fca4fffeab3d44e98ad2584f9f32a45a/artifacts/model
```
Note that the S3 URI of the logged model vary. 

Navigate to the MLflow service UI at [http://mlflow-server.local](http://mlflow-server.local),
and you should see your run under the experiment "mlflow-minio-test". You can browse the run parameters, metrics and artifacts. For example: 

* Training hyperparameters and evaluation metrics:

<img src="../images/mlflow-logging.png" width="1000"/>

You may notice that the "Metrics" and "Parameters" field are hidden by default, you can make them visible by clicking the "Columns" tab:

<img src="../images/mlflow-show-columns.png" width=1000 />

When clicking the Run Name, we can also check where the model and other related files have been uploaded:

<img src="../images/mlflow-uploaded-artifacts.png" width=1000 />

In this case, the model (which is a Pickle file) and its related files (such as the model dependency requirements) have been uploaded to the MinIO service. Navigate to [http://mlflow-minio-ui.local](http://mlflow-minio-ui.local) and login using "minioadmin" as both the username and password, we can see there is a bucket named "mlflow":

<img src="../images/minio-bucket-ui.png" width=1000 />

clicking the bucket (and its underlying folders) we can see the model and its related artifacts reside in the "mlflow" bucket:

<img src="../images/minio-model-artifacts.png" width=1000 />

* Finally, we can also see the model has been registered to MLflow:

<img src="../images/mlflow-model.png" width="1000"/>
